In [8]:
import yaml
from yaml.loader import SafeLoader, BaseLoader, FullLoader, UnsafeLoader

In [9]:
filepath = 'nasim/scenarios/benchmark/tiny.yaml'
with open(filepath, "r") as stream:
    try:
        curr_data = (yaml.safe_load(stream))
    except yaml.YAMLError as exc:
        print(exc)

In [10]:
#Print out representation of yaml data
print(curr_data)

{'subnets': [1, 1, 1], 'topology': [[1, 1, 0, 0], [1, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1]], 'sensitive_hosts': {'(2, 0)': 100, '(3, 0)': 100}, 'os': ['linux'], 'services': ['ssh'], 'processes': ['tomcat'], 'exploits': {'e_ssh': {'service': 'ssh', 'os': 'linux', 'prob': 0.8, 'cost': 1, 'access': 'user'}}, 'privilege_escalation': {'pe_tomcat': {'process': 'tomcat', 'os': 'linux', 'prob': 1.0, 'cost': 1, 'access': 'root'}}, 'service_scan_cost': 1, 'os_scan_cost': 1, 'subnet_scan_cost': 1, 'process_scan_cost': 1, 'host_configurations': {'(1, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat'], 'firewall': {'(3, 0)': ['ssh']}}, '(2, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat'], 'firewall': {'(1, 0)': ['ssh']}}, '(3, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat']}}, 'firewall': {'(0, 1)': ['ssh'], '(1, 0)': [], '(1, 2)': [], '(2, 1)': ['ssh'], '(1, 3)': ['ssh'], '(3, 1)': ['ssh'], '(2, 3)': ['ssh'], '(3, 2)': ['ssh']}, 'step_limit': 1000

In [11]:
#Edited the add subnet function to also update topology
def add_new_subnet(numOfHosts):
    curr_data['subnets'].append(numOfHosts)
    for i in range(0, len(curr_data['topology'])):
        curr_data['topology'][i].append(0)
    newRow = []
    for i in range(0, len(curr_data['topology'][0])):
        newRow.append(0)
    curr_data['topology'].append(newRow)
    
#Edited the add host function to also update both subnets and topology
def add_new_host_config(x, y, os, services, processes):
    address = '(' + str(x) + ', ' + str(y) + ')'
    curr_data['host_configurations'][address] = {'os': os, 'services': services, 'processes': processes}
    
    if x > len(curr_data['subnets']) - 1:
        add_new_subnet(1)
    elif y > curr_data['subnets'][x]:
        curr_data['subnets'][x] = curr_data['subnets'][x] + 1

In [12]:
#Edited the add subnet function to also update topology
def remove_subnet(subnetAddress):
    del curr_data['subnets'][subnetAddress]
    lenTop = len(curr_data['topology'])
    curr_data['topology'].remove(curr_data['topology'][lenTop - 1])
    lenRow = len(curr_data['topology'][0])
    for i in range(0, lenTop - 1):
        curr_data['topology'][i].remove(curr_data['topology'][i][lenRow - 1])

#Edited the remove host function to also update both subnets and topology
def remove_host_config(x, y):
    address = '(' + str(x) + ', ' + str(y) + ')'
    if curr_data['subnets'][x - 1] == 1:
        remove_subnet(x - 1)
    else:
        curr_data['subnets'][x - 1] -= 1
    
    del curr_data['host_configurations'][address]

In [13]:
#Allows for existing topology values to be updated
def edit_topology(x, y, value):
    curr_data['topology'][x][y] = value

In [14]:
print(curr_data['topology'])

[[1, 1, 0, 0], [1, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1]]


In [15]:
edit_topology(0, 2, 1)

In [16]:
print(curr_data['topology'])

[[1, 1, 1, 0], [1, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1]]


In [21]:
print(curr_data['host_configurations'])
print(curr_data['subnets'])
print(curr_data['topology'])

{'(1, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat'], 'firewall': {'(3, 0)': ['ssh']}}, '(2, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat'], 'firewall': {'(1, 0)': ['ssh']}}, '(3, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat']}}
[1, 1, 1]
[[1, 1, 1, 0], [1, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1]]


In [22]:
remove_host_config(3, 0)

In [23]:
print(curr_data['host_configurations'])
print(curr_data['subnets'])
print(curr_data['topology'])

{'(1, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat'], 'firewall': {'(3, 0)': ['ssh']}}, '(2, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat'], 'firewall': {'(1, 0)': ['ssh']}}}
[1, 1]
[[1, 1, 1], [1, 1, 1], [0, 1, 1]]


In [24]:
add_new_host_config(3, 0, 'linux', ['ssh'], ['tomcat'])

In [25]:
#When adding a new host, topologies are updated by adding 0's to the end of each layer(last index of each row)

print(curr_data['host_configurations'])
print(curr_data['subnets'])
print(curr_data['topology'])

{'(1, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat'], 'firewall': {'(3, 0)': ['ssh']}}, '(2, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat'], 'firewall': {'(1, 0)': ['ssh']}}, '(3, 0)': {'os': 'linux', 'services': ['ssh'], 'processes': ['tomcat']}}
[1, 1, 1]
[[1, 1, 1, 0], [1, 1, 1, 0], [0, 1, 1, 0], [0, 0, 0, 0]]
